# --- Day 15: Chiton --- 

https://adventofcode.com/2021/day/15

## Get Input Data

In [1]:
import numpy as np

In [2]:
def parse_data(filename):
    """Read in risk map data."""

    with open(f'../inputs/{filename}') as file:
        risk_map = [[int(x) for x in line.strip()] for line in file.readlines()]

    risk_map = np.matrix(risk_map)
    return risk_map

In [3]:
test_risk_map = parse_data('test_risk_map.txt')
test_risk_map

matrix([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
        [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
        [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
        [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
        [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
        [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
        [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
        [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
        [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
        [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [4]:
smaller_test_risk_map = test_risk_map[:3, :3]
smaller_test_risk_map

matrix([[1, 1, 6],
        [1, 3, 8],
        [2, 1, 3]])

In [5]:
# Get row slices
smaller_test_risk_map[:2, ...]

matrix([[1, 1, 6],
        [1, 3, 8]])

In [6]:
# Get column slices
smaller_test_risk_map[..., :2]

matrix([[1, 1],
        [1, 3],
        [2, 1]])

In [7]:
risk_map = parse_data('risk_map.txt')
risk_map

matrix([[6, 8, 9, ..., 4, 7, 4],
        [5, 3, 7, ..., 2, 9, 8],
        [8, 7, 9, ..., 8, 5, 1],
        ...,
        [9, 1, 2, ..., 8, 5, 6],
        [3, 7, 9, ..., 5, 8, 4],
        [9, 9, 8, ..., 8, 9, 2]])

In [8]:
risk_map.shape

(100, 100)

## Part 1
---

Will need to apply [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) to solve.

Chapter 7 in [Grokking Algorithms](https://www.manning.com/books/grokking-algorithms) provides an excellent tutorial on the algorithm -- highly recommend this book:

[![Grokking Algorithms](https://images.manning.com/360/480/resize/book/3/0b325da-eb26-4e50-8a2a-46042c647083/Bhargava-Algorithms_hires.png)](https://www.manning.com/books/grokking-algorithms)

Code for the algorithm can be found [here](https://github.com/egonSchiele/grokking_algorithms/blob/master/07_dijkstras_algorithm/python/01_dijkstras_algorithm.py).

<br>

This [blog post](https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html) from Udacity also gives an excellent overview/tutorial/code example of the algorithm.

In [43]:
def get_neighbors(position, map):
    """Return a list of [(y, x)] neighbor coordinates."""

    y, x = position[0], position[1]
    neighbors = [(y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    max_y, max_x = map.shape[0]-1, map.shape[1]-1

    # Filter out neighbors who have gone over the edge!
    good_neighbors = list(filter(lambda n: 0 <= n[0] <= max_y and 0 <= n[1] <= max_x, neighbors))

    return good_neighbors

In [44]:
def make_graph(map):
    """Convert a map into a dictionary/graph with appropriate weight/risk values for each node."""

    graph = {}

    for y in range(map.shape[0]):
        for x in range(map.shape[1]):
            neighbors = get_neighbors((y, x), map)
            
            graph[(y, x)] = {}
            for n in neighbors:
                if n != (0, 0):  # Don't add the starting node from neighbors lists
                    graph[(y, x)][n] = map[n]

            # print(f'({y}, {x}): neighbors: {neighbors}')

    # Set the "finish" node to have no neighbors
    graph[(map.shape[0]-1, map.shape[1]-1)] = {}

    return graph

In [45]:
smaller_test_risk_graph = make_graph(smaller_test_risk_map)
smaller_test_risk_graph

{(0, 0): {(1, 0): 1, (0, 1): 1},
 (0, 1): {(1, 1): 3, (0, 2): 6},
 (0, 2): {(1, 2): 8, (0, 1): 1},
 (1, 0): {(2, 0): 2, (1, 1): 3},
 (1, 1): {(2, 1): 1, (0, 1): 1, (1, 2): 8, (1, 0): 1},
 (1, 2): {(2, 2): 3, (0, 2): 6, (1, 1): 3},
 (2, 0): {(1, 0): 1, (2, 1): 1},
 (2, 1): {(1, 1): 3, (2, 2): 3, (2, 0): 2},
 (2, 2): {}}

In [47]:
def make_risk_table(map):
    """Return a hash table for every node on the map. Set the risk values to infinity, except
    for the first two neighbor nodes."""

    risk_table = {}

    for y in range(map.shape[0]):
        for x in range(map.shape[1]):
            risk_table[(y, x)] = float('inf')

    # # Get risk values for starting node:
    # start_neighbors = get_neighbors((0, 0), map)
    # for n in start_neighbors:
    #     risk_table[n] = map[n]

    # Then remove starting node from risk table
    # risk_table.pop((0, 0))

    # Set starting node risk value equal to zero
    risk_table[(0, 0)] = 0

    return risk_table

In [48]:
smaller_test_risk_table = make_risk_table(smaller_test_risk_map)
smaller_test_risk_table

{(0, 0): 0,
 (0, 1): inf,
 (0, 2): inf,
 (1, 0): inf,
 (1, 1): inf,
 (1, 2): inf,
 (2, 0): inf,
 (2, 1): inf,
 (2, 2): inf}

In [49]:
def find_lowest_risk_node(risk_table, processed_nodes):
    """ """
    lowest_risk = float("inf")
    lowest_risk_node = None
    
    # Go through each node.
    for node in risk_table:
        risk = risk_table[node]
        
        # If it's the lowest cost so far and hasn't been processed yet...
        if risk < lowest_risk and node not in processed_nodes:
            # ... set it as the new lowest-cost node.
            lowest_risk = risk
            lowest_risk_node = node

    return lowest_risk_node

In [50]:
find_lowest_risk_node(smaller_test_risk_table, [])

(0, 0)

In [51]:
def find_minimum_risk_path(risk_graph, risk_table):
    """ """
    processed_nodes = [(0, 0)]

    node = find_lowest_risk_node(risk_table, processed_nodes)
    
    while node is not None:
        # breakpoint()
        risk = risk_table[node]
    
        # Go through all the neighbors of this node.
        neighbors = risk_graph[node]

        for n in neighbors.keys():
            new_risk = risk + neighbors[n]

            # If it's less risky to get to this neighbor by going through this node...
            if risk_table[n] > new_risk:
                # ... update the risk for this node.
                risk_table[n] = new_risk

        # Mark the node as processed.
        processed_nodes.append(node)
        print(processed_nodes)
        # Find the next node to process, and loop.
        node = find_lowest_risk_node(risk_table, processed_nodes)

    end_node = max(risk_table.keys())

    return risk_table[end_node]

In [55]:
def find_minimum_risk_path2(risk_graph, risk_table):
    """ """

    unvisited_nodes = list(risk_table.keys())

    while unvisited_nodes:

        # The code block below finds the node with the lowest score
        current_min_node = None
        # KP: This seems very inefficient!
        for node in unvisited_nodes: # Iterate over the nodes
            if current_min_node == None:
                current_min_node = node
            elif risk_table[node] < risk_table[current_min_node]:
                current_min_node = node

        

        risk = risk_table[current_min_node]

        # Go through all the neighbors of this node.
        neighbors = risk_graph[current_min_node]

        for n in neighbors.keys():
            new_risk = risk + neighbors[n]

            # If it's less risky to get to this neighbor by going through this node...
            if new_risk < risk_table[n]:
                # ... update the risk for this node.
                risk_table[n] = new_risk

        unvisited_nodes.remove(current_min_node)

    end_node = max(risk_table.keys())

    return risk_table[end_node]

### Run on Test Data

In [37]:
smaller_test_risk_map

matrix([[1, 1, 6],
        [1, 3, 8],
        [2, 1, 3]])

In [53]:
find_minimum_risk_path2(smaller_test_risk_graph, smaller_test_risk_table)  # Should return 7

{(0, 0): 0, (0, 1): 1, (0, 2): 7, (1, 0): 1, (1, 1): 4, (1, 2): 12, (2, 0): 3, (2, 1): 4, (2, 2): 7}


7

In [54]:
test_risk_table = make_risk_table(test_risk_map)
test_risk_graph = make_graph(test_risk_map)
find_minimum_risk_path2(test_risk_graph, test_risk_table)  # Should return 40

{(0, 0): 0, (0, 1): 1, (0, 2): 7, (0, 3): 10, (0, 4): 17, (0, 5): 22, (0, 6): 23, (0, 7): 30, (0, 8): 34, (0, 9): 36, (1, 0): 1, (1, 1): 4, (1, 2): 12, (1, 3): 11, (1, 4): 14, (1, 5): 21, (1, 6): 23, (1, 7): 29, (1, 8): 32, (1, 9): 34, (2, 0): 3, (2, 1): 4, (2, 2): 7, (2, 3): 13, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 23, (2, 8): 25, (2, 9): 33, (3, 0): 6, (3, 1): 10, (3, 2): 16, (3, 3): 17, (3, 4): 26, (3, 5): 22, (3, 6): 21, (3, 7): 26, (3, 8): 31, (3, 9): 38, (4, 0): 13, (4, 1): 14, (4, 2): 20, (4, 3): 20, (4, 4): 24, (4, 5): 23, (4, 6): 28, (4, 7): 27, (4, 8): 28, (4, 9): 29, (5, 0): 14, (5, 1): 17, (5, 2): 18, (5, 3): 27, (5, 4): 25, (5, 5): 25, (5, 6): 33, (5, 7): 28, (5, 8): 31, (5, 9): 36, (6, 0): 15, (6, 1): 18, (6, 2): 23, (6, 3): 32, (6, 4): 34, (6, 5): 26, (6, 6): 28, (6, 7): 32, (6, 8): 33, (6, 9): 34, (7, 0): 18, (7, 1): 19, (7, 2): 21, (7, 3): 26, (7, 4): 30, (7, 5): 28, (7, 6): 29, (7, 7): 35, (7, 8): 36, (7, 9): 43, (8, 0): 19, (8, 1): 21, (8, 2): 30, (8, 3): 29, 

40

### Run on Input Data

In [56]:
risk_table = make_risk_table(risk_map)
risk_graph = make_graph(risk_map)
find_minimum_risk_path2(risk_graph, risk_table)

KeyboardInterrupt: 

In [ ]:
import sys
 
class Graph(object):
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        '''
        This method makes sure that the graph is symmetrical. In other words, if there's a path from node A to B with a value V, there needs to be a path from node B to node A with a value V.
        '''
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
        
        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value
                    
        return graph
    
    def get_nodes(self):
        "Returns the nodes of the graph."
        return self.nodes
    
    def get_outgoing_edges(self, node):
        "Returns the neighbors of a node."
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        "Returns the value of an edge between two nodes."
        return self.graph[node1][node2]

In [ ]:
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
 
    # We'll use this dict to save the cost of visiting each node and update it as we move along the graph   
    shortest_path = {}
 
    # We'll use this dict to save the shortest known path to a node found so far
    previous_nodes = {}
 
    # We'll use max_value to initialize the "infinity" value of the unvisited nodes   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0   
    shortest_path[start_node] = 0
    
    # The algorithm executes until we visit all nodes
    while unvisited_nodes:
        # The code block below finds the node with the lowest score
        current_min_node = None
        for node in unvisited_nodes: # Iterate over the nodes
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
                
        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node
 
        # After visiting its neighbors, we mark the node as "visited"
        unvisited_nodes.remove(current_min_node)
    
    return previous_nodes, shortest_path

In [ ]:
dijkstra_algorithm(risk_graph, (0, 0))

## Part 2
---

### Run on Test Data

### Run on Input Data